In [31]:
import numpy as np
import pandas as pd

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
df = pd.read_csv("../../data/.csv")
df.head(5)

,Unnamed: 0,shixin_id,duty
0,1,2905,被告上海纺友贸易有限公司应于本判决生效后十日内支付原告江苏盛虹面料有限公司货款人民币 元如...
1,6,3650,一被告上海栩瑞电气设备有限公司确认尚欠原告上海宝临超亚电器有限公司货款 元 原告上海宝临超亚...
2,16,11617,原告上海玖峰数码科技有限公司于本判决生效之日起十日内支付被告张宁顺违法解除劳动合同赔偿金 元...
3,21,15642,被执行人于年月前向申请执行人支付工资人民币万元 被执行人于年月前向申请执行人支付工资人民币万...
4,22,16229,一原告上海顶天钢铁发展有限公司与被告内蒙古兴托物资有限公司于年月日签订的 内蒙古托县至兴和煤...


In [10]:
df.shape

(17255, 3)

In [11]:
# df = df.sample(2000)
# Commenting this out to perform on all the entries; 

In [12]:
import jieba

In [13]:
# Need to be length greater than 10;

def length_bigger_than_10(text):
    if len(text)>=10:
        return True
    else:
        return False


#新建length列
df['length'] = df.agg({'duty': length_bigger_than_10})
#根据length列来保留长度大于10的文本，并将其放入新的dataframe中
df = df[df['length']==True]

In [14]:
import nltk
import jieba

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/leijunjie/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [16]:
#停止词
from nltk.corpus import stopwords
stopwords = stopwords.words("chinese")



def clean_text(text):
    wordlist = jieba.lcut(text) 
    #去除停用词和长度小于2的词语
    wordlist = [w for w in wordlist if w not in stopwords and len(w)>2]
    #将中文数据组织成类似西方于洋那样，词语之间以空格间隔
    document =  " ".join(wordlist)
    return document


#使用dataframe.agg对content列实行clean_text方法
df['duty'] = df.agg({'duty': clean_text})
corpus = df['duty']
corpus.head()

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/ys/98s0q77d0bl4mpqsp4lf7c740000gn/T/jieba.cache
Loading model cost 1.417 seconds.
Prefix dict has been built successfully.


0                                         第二百 二十九条 负之款
1                               电气设备 原被告 第一期 电气设备 赔偿损失
2                                   数码科技 劳动合同 赔偿金 数码科技
3                                         工资卡 汇丰银行 第一笔
4    内蒙古 内蒙古 内蒙古 内蒙古 保证金 内蒙古 内蒙古 高速公路 内蒙古 第二百 二十九条 ...
Name: duty, dtype: object

In [17]:
## jieba 分词包

def chinese_word_cut(mytext):
    return " ".join(jieba.cut(mytext))

In [18]:
df["duty_cutted"] = df.duty.apply(chinese_word_cut)

In [19]:
df.duty_cutted.head()

0                                     第二百   二十九条   负之款
1                       电气设备   原被告   第一期   电气设备   赔偿损失
2                             数码科技   劳动合同   赔偿金   数码科技
3                                     工资卡   汇丰银行   第一笔
4    内蒙古   内蒙古   内蒙古   内蒙古   保证金   内蒙古   内蒙古   高速公路...
Name: duty_cutted, dtype: object

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [21]:
stpwrdpath = "../../data/stopwords.txt"
stpwrd_dic = open(stpwrdpath, 'rb')
stpwrd_content = stpwrd_dic.read().decode('utf-8')#将停用词表转换为list
stpwrdlst = stpwrd_content.splitlines()
stpwrd_dic.close()

In [22]:
n_features = 1000

In [23]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                #stop_words='english',
                                stop_words=stpwrdlst,
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(df.duty_cutted)

In [24]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                #stop_words='english',
                                stop_words=stpwrdlst,
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(corpus)

In [25]:
from sklearn.decomposition import LatentDirichletAllocation

In [26]:
lda = LatentDirichletAllocation(n_components = 15,max_iter=50,
                                learning_method='online',
                                learning_offset=50,
                                random_state=0)

In [27]:
lda.fit(tf)

LatentDirichletAllocation(learning_method='online', learning_offset=50,
                          max_iter=50, n_components=15, random_state=0)

In [28]:
import pyLDAvis 
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
14    -0.107863 -0.075894       1        1  8.952319
1     -0.056558 -0.025925       2        1  8.475751
13    -0.038371 -0.229837       3        1  8.065720
8     -0.090309 -0.057126       4        1  7.871470
2      0.007292 -0.011188       5        1  7.818651
11     0.024273 -0.216562       6        1  7.031485
6      0.244009  0.032288       7        1  6.763765
10     0.258112  0.077931       8        1  6.680496
5     -0.136870  0.266172       9        1  6.485205
9     -0.036346 -0.010465      10        1  6.140172
3     -0.096842 -0.060244      11        1  5.972129
0     -0.041832 -0.015860      12        1  5.433498
7     -0.147026  0.256547      13        1  5.008344
12    -0.043320 -0.008619      14        1  4.656381
4      0.261651  0.078781      15        1  4.644615, topic_info=     Term         Freq        Total Category  logprob  loglift
283   房地产  1707.000000  1707.000000  Default  30.0000  30.0000
178  国际贸易   674.000000   674.000000  Default  29.0000  29.0000
2     一次性   659.000000   659.000000  Default  28.0000  28.0000
211  小额贷款   734.000000   734.000000  Default  27.0000  27.0000
248  建设工程   544.000000   544.000000  Default  26.0000  26.0000
..    ...          ...          ...      ...      ...      ...
428   第三条    19.361161    20.291109  Topic15  -4.8721   3.0225
24    专利号    16.948345    17.878291  Topic15  -5.0052   3.0160
455   维护费    11.054683    11.984630  Topic15  -5.4326   2.9887
512   赔偿金   196.234691   365.097345  Topic15  -2.5561   2.4486
116  劳动合同    87.639765   573.460084  Topic15  -3.3622   1.1910

[505 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
2         8  0.997845   一次性
4        10  0.991468   丁莉萍
5        10  0.985383  万分之一
6         3  0.991186  万分之三
7        13  0.979623  万分之二
...     ...       ...   ...
580       5  0.989379   黄久聪
581      14  0.978166   黄志强
582       5  0.989753   黄碧莺
583      14  0.959010   黄雁玲
584       5  0.988270   黄齐杖

[485 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 2, 14, 9, 3, 12, 7, 11, 6, 10, 4, 1, 8, 13, 5])

In [29]:
p = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
# pyLDAvis.save_html(p, 'lda_10_sh_all.html')

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
